In [26]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import json
pd.set_option("display.max_rows",500) 
pd.set_option("display.max_columns",500) 
pd.set_option('display.max_colwidth', None)
import requests
import json
import time
import requests
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests_cache
from retry_requests import retry
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from xgboost import plot_importance

# Güneş Enerjisi Verilerinin Çekilmesi (EPİAŞ)

Bu başlık altındaki API ile çekilen veri güneş_enerji_verileri klasöründe hazır olarak sunulmuştur. (gunesenerji_uretim.csv)

In [ ]:
url = "https://giris.epias.com.tr/cas/v1/tickets"
payload = {
    'username': 'your_username',
    'password': 'your_password'
}
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'text/plain'
}
response = requests.post(url, data=payload, headers=headers)

if response.status_code == 201:
    tgt = response.text
    print("TGT:", tgt)
else:
    print("TGT alınırken hata oluştu:", response.status_code)
    print("Yanıt:", response.text)

In [ ]:
url = "https://seffaflik.epias.com.tr/electricity-service/v1/renewables/data/licensed-realtime-generation"

headers = {
    "Accept-Language": "en",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "TGT": tgt
}
start_date = datetime(2022, 1, 1)
end_date = datetime(2025, 3, 2)

all_data = []

current_start = start_date
while current_start < end_date:
    current_end = current_start + relativedelta(months=1)

    payload = {
        "startDate": current_start.isoformat() + "+03:00",
        "endDate": current_end.isoformat() + "+03:00"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print(f"İstek: {payload['startDate']} - {payload['endDate']}")

    if response.status_code == 200:
        data = response.json()
        all_data.append(data)
        print("Başarılı")
    else:
        print("Hata:", response.status_code)
        print(response.text)

    current_start = current_end

df_gunes=pd.DataFrame(all_data[0]["items"])
df_gunes=df_gunes[["date","gunes"]]

In [ ]:
all_items = []
for entry in all_data:
    items = entry.get("items", [])
    all_items.extend(items)

df_gunes = pd.DataFrame(all_items)
df_gunes=df_gunes[["date","gunes"]]
df_gunes.to_csv("güneş_enerji_verileri/gunesenerji_uretim.csv", index=False)

# Kurulu Güç Verilerinin Çekilmesi (EPİAŞ)

Bu başlık altındaki API ile çekilen veri güneş_enerji_verileri klasöründe hazır olarak sunulmuştur. (epias_kuruluguc.csv)

In [ ]:
url = "https://seffaflik.epias.com.tr/electricity-service/v1/renewables/data/renewables-participant"

headers = {
    "Accept-Language": "en",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "TGT": tgt
}

years = [2022, 2023, 2024, 2025]
all_data = []

for year in years:
    payload = {"year": str(year)}
    response = requests.post(url, headers=headers, data=json.dumps(payload))

    print(f"🔄 {year} verisi isteniyor...")

    if response.status_code == 200:
        data = response.json()
        all_data.append(data)
        print(f"{year} verisi başarıyla çekildi.")
    else:
        print(f"{year} verisi çekilemedi. Kod: {response.status_code}")
        print("Hata mesajı:", response.text)

In [32]:
all_items = []
for entry in all_data:
    items = entry.get("items", [])
    all_items.extend(items)

df_kuruluguc = pd.DataFrame(all_items)
df_kuruluguc=df_kuruluguc[df_kuruluguc["facilityType"]=="Fotovoltaik"]
df_kuruluguc=df_kuruluguc[["orgName","facilityName","installedPower","entryDate"]]
df_kuruluguc.drop_duplicates(inplace=True)
df_kuruluguc = df_kuruluguc.loc[df_kuruluguc.groupby(["orgName", "facilityName"])["installedPower"].idxmax()].reset_index(drop=True)
df_kuruluguc_il= pd.read_excel("güneş_enerji_verileri/il_kuruluguc.xlsx")
df_kuruluguc=pd.merge(df_kuruluguc,df_kuruluguc_il,on=["facilityName"],how="left")
df_kuruluguc.to_csv("güneş_enerji_verileri/epias_kuruluguc.csv", index=False)

# Hava Durumu Verilerinin Çekilmesi (OpenMeteo)

Bu başlık altındaki API ile çekilen veri güneş_enerji_verileri klasöründe hazır olarak sunulmuştur. (weather.csv)

In [ ]:
df_coordinates=pd.read_csv("güneş_enerji_verileri\Coordinates.csv")
df_coordinates.head()

In [ ]:
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = Client(session=retry_session)

start_date = datetime(2022, 1, 1)
end_date = datetime(2025, 3, 2)
step = timedelta(days=90)

all_city_data = []

for index, row in df_coordinates.iterrows():
    city_name = row["Province"]
    lat = row["latitude"]
    lon = row["longitude"]
    
    current_start = start_date

    while current_start < end_date:
        current_end = min(current_start + step, end_date)

        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": current_start.strftime("%Y-%m-%d"),
            "end_date": current_end.strftime("%Y-%m-%d"),
            "hourly": [
                "temperature_2m",
                "cloud_cover",
                "is_day",
                "sunshine_duration",
                "rain",
                "snowfall",
                "global_tilted_irradiance_instant"
            ]
        }

        success = False
        retry_count = 0
        max_retries = 5

        while not success and retry_count < max_retries:
            try:
                responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
                response = responses[0]
                hourly = response.Hourly()

                df = pd.DataFrame({
                    "date": pd.date_range(
                        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                        freq=pd.Timedelta(seconds=hourly.Interval()),
                        inclusive="left"
                    ),
                    "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
                    "cloud_cover": hourly.Variables(1).ValuesAsNumpy(),
                    "is_day": hourly.Variables(2).ValuesAsNumpy(),
                    "sunshine_duration": hourly.Variables(3).ValuesAsNumpy(),
                    "rain": hourly.Variables(4).ValuesAsNumpy(),
                    "snowfall": hourly.Variables(5).ValuesAsNumpy(),
                    "global_tilted_irradiance_instant": hourly.Variables(6).ValuesAsNumpy(),
                })
                df["city"] = city_name
                df["latitude"] = lat
                df["longitude"] = lon

                all_city_data.append(df)
                print(f"{city_name}: {current_start.date()} - {current_end.date()} verisi alındı.")
                success = True
            except Exception as e:
                retry_count += 1
                print(f"{city_name}: {current_start.date()} - {current_end.date()} verisi alınamadı. {retry_count}. deneme. Hata: {e}")
                time.sleep(60)

        current_start = current_end
        time.sleep(1)

combined_df = pd.concat(all_city_data, ignore_index=True)

In [ ]:
combined_df["date"] = combined_df["date"].dt.tz_convert("Europe/Istanbul")
combined_df.to_csv("güneş_enerji_verileri/weather.csv", index=False)

# Hava Durumu ve Kurulu Güç Verilerinin Birleştirilmesi

In [ ]:
weather=pd.read_csv("güneş_enerji_verileri/weather.csv")
weather.head()

In [ ]:
df_kuruluguc=pd.read_csv("güneş_enerji_verileri/epias_kuruluguc.csv")
df_kuruluguc.head()

In [11]:
df_kuruluguc["entryDate"] = pd.to_datetime(df_kuruluguc["entryDate"])
df_kuruluguc["entryDate"]=df_kuruluguc["entryDate"].dt.date
expanded_rows = []
for _, row in df_kuruluguc.iterrows():
    date_range = pd.date_range(start="2022-01-01", end="2025-03-02", freq="D")
    for date in date_range:
        new_row = row.copy()
        new_row["entryDate"] = date
        expanded_rows.append(new_row)

df_kuruluguc_expanded = pd.DataFrame(expanded_rows)

In [ ]:
df_kuruluguc_expanded=df_kuruluguc_expanded[(df_kuruluguc_expanded["entryDate"] >= pd.to_datetime("2022-01-01")) & (df_kuruluguc_expanded["entryDate"] <= pd.to_datetime("2025-03-01"))]
df_kuruluguc_expanded["entryDate"]=df_kuruluguc_expanded["entryDate"].dt.date
df_kuruluguc_expanded2 = df_kuruluguc_expanded.groupby(["entryDate", "city"], as_index=False)["installedPower"].sum()
df_kuruluguc_expanded2.head()

In [13]:
weather['date'] = pd.to_datetime(weather['date'])
weather['Year'] = weather['date'].dt.year
weather['Month'] = weather['date'].dt.month
weather['Day'] = weather['date'].dt.day

df_kuruluguc_expanded2['entryDate'] = pd.to_datetime(df_kuruluguc_expanded2['entryDate'])
df_kuruluguc_expanded2['Year'] = df_kuruluguc_expanded2['entryDate'].dt.year
df_kuruluguc_expanded2['Month'] = df_kuruluguc_expanded2['entryDate'].dt.month
df_kuruluguc_expanded2['Day'] = df_kuruluguc_expanded2['entryDate'].dt.day

weather = weather.merge(df_kuruluguc_expanded2[['Year', 'Month', 'Day', 'installedPower', 'city']], left_on=['Year', 'Month', 'Day','city'], right_on=['Year', 'Month', 'Day','city'], how='left').drop(columns=['Day',"Year","Month","latitude","longitude"])
weather=weather[weather["installedPower"].notnull()]

In [ ]:
weather["panel_temprature"]=weather["temperature_2m"]+(25*weather["global_tilted_irradiance_instant"]/800)
#Tp = To + (NOTC-20C)* It / 800 (NOCT (Nominal Operating Cell Temperature) --> panelin kendi sıcaklığını hesaplayan formül)

In [ ]:
weather["productivity_per"]=weather["panel_temprature"].apply(lambda x: 1-((x-25)*0.004) if x>=25 else 1)
# 25 derecenin üzerinde her bir derece için güneş panellerinin verimliliği %0.04 düşer.
weather["productivity"]=weather["installedPower"]*weather["productivity_per"]

In [ ]:
weather["non_cloud"] = 100 - weather["cloud_cover"]
weather["weather_relation"] = weather["productivity"]*weather["non_cloud"]*weather["sunshine_duration"]*weather["is_day"]/weather["productivity"].sum()/3600
# weather_relation değişkeni hava durumu verisinden yararlanarak oluşturulan değişkendir.

# Verinin Modele Hazırlanması

In [ ]:
df=pd.read_csv("güneş_enerji_verileri/gunesenerji_uretim.csv")
df=df[["date","gunes"]]
df["date"]=pd.to_datetime(df["date"])
df.drop_duplicates(inplace=True)
df["gunes"] = df["gunes"].fillna(method="ffill")
df['gunes_1g'] = df['gunes'].shift(24)
df.dropna(inplace=True)
start_date = '2022-01-01'
end_date = '2025-03-01'
mask = (df['date'] >= start_date) & (df['date'] <= end_date)
df = df[mask]
df.head()

In [18]:
df["hour"]=df["date"].dt.hour #onehot encoding!!!
df["dayofweek"]=df["date"].dt.dayofweek #onehot encoding!!!
df["month"]=df["date"].dt.month #onehot encoding!!!

In [ ]:
il_sum = weather.groupby('date')['weather_relation'].sum().reset_index()
il_sum["date"]=pd.to_datetime(il_sum['date'])
df = pd.merge(df, il_sum, on='date', how='left')
df.dropna(inplace=True)

In [20]:
def season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    elif month in [9, 10, 11]:
        return "autumn"

df["season"]=df["date"].dt.month.apply(season) #onehot encoding!!!

In [ ]:
df=pd.get_dummies(df, columns=["hour","dayofweek","season","month"])
df.head()

In [22]:
train = df[(df["date"] >= "2022-01-01") & (df["date"] < "2024-07-01")]
valid = df[(df["date"] >= "2024-07-01") & (df["date"] < "2025-01-01")]
test = df[df["date"] >= "2025-01-01"]
X_train, y_train = train.drop(columns=["date", "gunes"]), train["gunes"]
X_valid, y_valid = valid.drop(columns=["date", "gunes"]), valid["gunes"]
X_test,  y_test  = test.drop(columns=["date", "gunes"]),  test["gunes"]

# Model Eğitimi

In [ ]:
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    colsample_bytree=1.0,
    gamma=0.1,
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100,
    reg_alpha=0.1,
    reg_lambda=1.5,
    subsample=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    early_stopping_rounds=20,
    verbose=True
)

predictions = model.predict(X_test)
predictions = np.where(predictions < 0, 0, predictions)

mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'MAE: {mae:.4f}, R²: {r2:.4f}')

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(y_test.values, label='Gerçek Değerler', linewidth=2)
plt.plot(predictions, label='Tahmin Edilen Değerler', linewidth=2)
plt.title('Gerçek vs Tahmin Edilen Değerler')
plt.xlabel('Gözlem')
plt.ylabel('Değer')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plot_importance(model, importance_type='gain', max_num_features=20)
plt.title("XGBoost Feature Importance (Gain)")
plt.tight_layout()
plt.show()